# Job Market Assistant


## Initialization


Get _.env_ and _gemini.json_ files from the maintainers and copy them to the root of the project.


In [17]:
!python -m venv venv

In [18]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [14]:
# Using Ruff as a formater; most of the times line length is exceed because of comments or links
!flake8 core --ignore E501

In [15]:
# Cannot type imports from third-party libraries
!mypy core --python-executable $(which python) --disable-error-code='import-untyped'

In [6]:
!pip install python-dotenv

In [1]:
import time

import pandas as pd
import vertexai
from dotenv import load_dotenv
from google.oauth2.service_account import Credentials
from IPython.display import Markdown
from linkedin_jobs_scraper.query import Query, QueryOptions

from core import linkedin_scraper
from core.job_market_assistant import JobMarketAssistant
from core.models import ChatGPT, Gemini, Llama

# ChatGPT, Llama
load_dotenv()


# Gemini
credentials = Credentials.from_service_account_file("gemini.json")
vertexai.init(
    project="linkedln-advisor", location="europe-central2", credentials=credentials
)

/home/krzychu/anaconda3/envs/nlp_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Framework

Our class-based structure allows to encapsulate reused code and perform experiments easily by adding new models and altering the prompts.

All the models have to implement _complete_ method. With parameter _debug_ set to _True_ the underlying querying history is visible. You may want to use the parameter to observe the innerworkings of the system.


In [2]:
debug_chat_gpt = ChatGPT(debug=True)
completion = debug_chat_gpt.complete("Hello Chat!")

>>> ChatGPT: Hello Chat!
ChatGPT >>>: Hello there [username]! How are you doing?


In [3]:
debug_gemini = Gemini(debug=True)
completion = debug_gemini.complete("Hello Gemini!")

>>> Gemini: Hello Gemini!
Gemini >>>: Hello! I am glad you are here. What can I do for you today?


In [4]:
debug_llama = Llama(debug=True)
completion = debug_llama.complete("Hello Llama!")

>>> Llama: Hello Llama!
Llama >>>: Hello there! I'm here to help with any questions or tasks you may have. What's on your mind today?


| Model   | Version                |
| ------- | ---------------------- |
| ChatGPT | gpt-3.5-turbo-instruct |
| Gemini  | gemini-1.0-pro-002     |
| Llama   | llama3-8b              |


In [5]:
chat_gpt = ChatGPT()
gemini = Gemini()
llama = Llama()

The system uses 4 different prompts each serving unique purpose and having a distinct set of available parameters:

| Method             | Parameter             | Purpose                                                                      | Parameters                                 |
| ------------------ | --------------------- | ---------------------------------------------------------------------------- | ------------------------------------------ |
| get_cover_letter   | cover_letter_prompt   | Generate a cover letter based on the job posting and CV.                     | job_title, company, job_desc, location, cv |
| get_missing_skills | job_title_prompt      | Generate the desired _job_title_ from the CV unless provided explicitely     | cv                                         |
| get_missing_skills | key_points_prompt     | Summarize requirements for a position before feeding them to the next prompt | job_title, company, job_desc, location     |
| get_missing_skills | missing_skills_prompt | Identify skill gaps for the job title and return them as a list              | key_points, cv                             |


In [9]:
job_market_assistant = JobMarketAssistant(
    model=chat_gpt,
    cover_letter_prompt="Write a cover letter for this job. Do not use too long and unnecessarily sophisticated words.\n\nMy CV:\n{cv}\n\nJob Posting:\n{job_desc}",
    job_title_prompt="Summarize the job title based on the following CV in maximum 10 words:\n\n{cv}\n\nProvide only the job title as plain text without any quotes and punctuation.",
    key_points_prompt="Job Posting:\n\n{job_desc}\n\nGenerate key points from these job posting (focus on skils and requirements) sorted from the most important to the least?",
    missing_skills_prompt="This is my CV:\n\n{cv}\n\nReturn me list of skills that I should learn and I don't have in my CV to become more employable for jobs with requirements listed below sorted from the most important to the least. The list has to be in Markdown format. Job postings:\n\n",
)

## Data sources


The job postings may be provided directly or scraped from LinkedIn using one of the helper methods.


In [7]:
file_job_postings = pd.read_feather("data/job_postings.feather")
file_job_postings

,job_title,company,job_desc,location
39,Applied Data Scientist,dunnhumby,dunnhumby is the global leader in Customer Dat...,"London, England, United Kingdom"
47,Data Scientist,Builder.ai,Description\n\nAbout Builder.ai\n\nWe’re on a ...,"London, England, United Kingdom"
267,Machine Learning Engineer - ML Ops,Builder.ai,Description\n\nAbout Builder.ai\n\nWe’re on a ...,"London, England, United Kingdom"
147,"Software Development Engineer, Caching",myGwork - LGBTQ+ Business Community,This inclusive employer is a member of myGwork...,"Dublin, County Dublin, Ireland"
265,Machine Learning Engineer,Stackfix,"About Us\n\nAt Stackfix, we’re transforming ho...","London, England, United Kingdom"
...,...,...,...,...
183,Sales Business Analyst,Aer Lingus,Your role\n\nAre you ready to make a significa...,"Dublin, County Dublin, Ireland"
20,"Software Development Engineer, AWS Security Se...",myGwork - LGBTQ+ Business Community,This inclusive employer is a member of myGwork...,"Dublin, County Dublin, Ireland"
81,Application & Service Engineer,Altus Group Ltd,I am pleased to announce that we are looking f...,"Cork, County Cork, Ireland"
327,Graduate Data Scientist,Worldpanel by Kantar,"We’re the world’s leading data, insights, and ...","Greater London, England, United Kingdom"


In [2]:
url_job_postings = linkedin_scraper.scrap_urls(
    (
        "https://www.linkedin.com/jobs/view/3937215366/?alternateChannel=search&refId=pyO8PX6NFy5hyGNWQ5uRxQ%3D%3D&trackingId=aNQSGBRWWobujlSYEpbvaQ%3D%3D",
        "https://www.linkedin.com/jobs/view/3931218091/?alternateChannel=search&refId=%2BwGMIZDUEBj8w%2BcBBmKX0g%3D%3D&trackingId=MKGRxpSmYvT7yYfc0vnezg%3D%3D",
        "https://www.linkedin.com/jobs/view/3939354205/?alternateChannel=search&refId=jzm6ENFRWaolwGXJgicMTw%3D%3D&trackingId=Ryft9wMmyemu5LZP3YMVkQ%3D%3D&trk=d_flagship3_search_srp_jobs",
        "https://www.linkedin.com/jobs/view/3933974972/?alternateChannel=search&refId=jzm6ENFRWaolwGXJgicMTw%3D%3D&trackingId=Nf75Lj8H45f%2F3Wn0YZWGow%3D%3D&trk=d_flagship3_search_srp_jobs",
        "https://www.linkedin.com/jobs/view/3706100131/?eBP=CwEAAAGP2nLSS1cVH9CpQXLJCQYoBELJe_DMnb_FF0wnPFmYKZ8RevBvThpD_ekhE_mtBIX8fy5hK5ayxs7uQI8M551BR4OaZ0Vk6wIa_yi-E6P_orjzE4PKlU6nMCHwWnQoskuS4a0jCI8057qk3wYLKiI8pZ_mITVGGc_tOrpA5WVKMj_Zeoa44P2nxT1vJ1KUFmyeIiSBOAnIdhZu4fTCqmELBiPkEAD5eKftXGAuDr3pbYOWdt3TZ9lCMf-xY0rQGFaYih57c03_fADVuhjZygq2VkyoJpNIwGj_5INh7nT_fjMLCs_1POakqPL3o3-H7t2RAdFyjMtjYZ0D3_2500LfftGIcnS0Rrjl9Xss1xgu1HcRoGWs-Yx0hkoolePWe-KCNgbczGhVVMQkW3TEwVYwNmq-5ucGIuEvbNhqYo9H2th1MA7NLumAVxtEIWH6IK8TLe7JiAhf8VBblyQm2RgHaZTO&refId=ejemPX0qrd8qnaCrjBlNWw%3D%3D&trackingId=nRFdwwj3o5%2B%2B7KFMCCFo1w%3D%3D&trk=flagship3_search_srp_jobs",
        "https://www.linkedin.com/jobs/view/3926449844/?eBP=CwEAAAGP2nOgU8_sfnyPI_cLUth9VbgP3dIs3RClQtIwUPjlvFsnHuQAiIaEqWo-DXN0PMr8OSoQIj3obkACHeuwBugF67GnD_Z_IL8WkR2I8yAYJM3taMePkxBmPwI2zsd6facjFtS8XDaha22A0tXYebVPQscG8pvSd5vZSzyVY3r4GN9iBpOrZsgnD6Ql3f-o0NpW4H_Ea-GRZoVD5abwhA_FkiQjPW1_JL_b-MXYWhVlPXImXCYdh2s86zuIfyhmxYCto6nfZoLqgBGfNGa_g5NXMP7cF4tj57CaZxE2Bm22qe_vCdTfkUoiwWNVUa3rSEEtzB49dimLquOca5B2-Sa8IsYSJFTcpWw4GOInhexoSnI-Uawuu1AH_BBLQ-NqGXSAmpoIpbQKtCSID8PnSgUTJx_k4htjYF7FrG7ciSLAhVULZSLkCQ-UNYj9ubHhOzjuGpE1zvNnNKflzOLkmpXUiMg5USw&refId=NB7IjF%2F4NI3gjXiPX1PlkQ%3D%3D&trackingId=D9AGbek05Cnb7pxMYxiqpg%3D%3D&trk=flagship3_search_srp_jobs",
        "https://www.linkedin.com/jobs/view/3904363506/?eBP=CwEAAAGP2nOgU4zBnhBLcSx9mV0FYGVGgNyQ1sxXETXxVxPVFk53FLqXR9gH9f4U2Nqac6fmAWSBzAAnCbbBO-YY8F0YdhOwtuIQY01HY_W4sWI49I8PJP6pBSKCHPSDOj4lmCGZlDuy8LguqiBqKKAvNjUMPRw6yWT83z-Xo4YwNA62ju020hTZ-YFlK-WATuOZGhVoK0iwbySo89vy-cf2j5ZP7vpLrArpWUtzjNrGCItD_qOoCubYiF_93A_axtk8RkbmwmTIxL7AOl5Ql_veoGeNXdfdiw5wDLiGZBwp1lxvOIFiC_X7655cFQqm5jhB8p7jg8bRSYKSwbawikJntoTMDxs6TpvqjYHnbsT9xKaaPOpgDes3A4U3JONRtNa5S9y_-C3v9QWMC1DwwuPAqcETf7Llf47r4Npo-dxM17YgHj3FQicUzkI8wYSKpmzU0uK44dASHScYXwN5TWVX3pZxqEprZNY&refId=NB7IjF%2F4NI3gjXiPX1PlkQ%3D%3D&trackingId=vI35XF233bfRnGy6BAE%2FSQ%3D%3D&trk=flagship3_search_srp_jobs",
        "https://www.linkedin.com/jobs/view/3910213820/?eBP=CwEAAAGP2nOgU4o3HMYGtDJlo8CcLUUFwd-jc9D9_hbJ5mLrtYZcutRbb5mRXvtMJ1Ig92XZ0FOxY5SkMcrAfg3VK1tY1JObDFpgTe_y9Y7iNKbX94lMzuYnYBJn21Eeymf77lmdICyU6QH-yPQbdcf33E72AC_xt7EBxnpOv9M0sGXqJc11puvSVQGbXpW5W5Sbbsv6QTzs9FxlovRKu6pEPDCCVzjrHrC9d_gOylePjmUOnabDFkppFa3gbB8zh_il2KSTYB_lhCtl1Kxi-5y0NcsECFB5OR5KFUr0npWo7btVAewY5QBeyUFcbcX2O9pXjVdImXT40xQBaQbIUbZXEZeSEaFroWyIc-kB8FzaKMlLmIK5ezXmgfdGx_f10lWV_9xSFVlvaWI1MeG5adcsO0fTM2fY0ucITXYFS2qbispOnegSv3cS_p4WqGTNkLxg_Nxjr8ppKq8HMMZud8dppz5Izp2lOyg&refId=NB7IjF%2F4NI3gjXiPX1PlkQ%3D%3D&trackingId=1OHZffvErAg6ayGGSMaGSQ%3D%3D&trk=flagship3_search_srp_jobs",
        "https://www.linkedin.com/jobs/view/3934612685/?eBP=CwEAAAGP2nOgUxurolZNvR-9t8O15gg8Hg73vyxdiM7INtIvQbruj1Lad7exeumIa4mrXe73RR3QJ3t-RkVakkFZqmPBu1QNpw4oyXWLO0A0d7RG_uAQJ0YJCdq34tT2c50_4HPgem5RvZ0Q_KcjsPUZuVDkiy0i5xxjusB-K9fzpJ1kHeoLJVsY4-Hdgjrd8hdvpQ6XwtPErHWtPsHmaBPgMvJfGw22Nu_SW6Or4WeOmLmaBroXMpeJGuVz7UqETkN7ro6h77j_Rsi25h8wYeL7S-NY6zlt5HCltq-cIXuFW_kpQ_FXPqHvnKuXkKpKchCBqgCSa3AUTCEaNF60I50pHS9xNHnhro0RaMODYkEAB1YhKBDLW_JY_qWIORfrs48oEWG5HCt_yCwEQCzO6ydupfJkjLOgCjEpOc3wFSMK6knh7yySP4zbSOArdmbPkMsaTTvgFHCG3loj9FRN49Dbrl5RFd1C3rg&refId=NB7IjF%2F4NI3gjXiPX1PlkQ%3D%3D&trackingId=xDb049gA1gGIWmWSmZ6jfg%3D%3D&trk=flagship3_search_srp_jobs",
        "https://www.linkedin.com/jobs/view/3903916300/?eBP=CwEAAAGP2nWWOL08uhH7U-M5C0J0CphPf6gFN4SMMBAXxYf-YdOwMoRtXj0UBwSmzeTRcrgSE8fpf7yZ0xy0_rHcDoJxAtbx85i8FJ3WTzcEZNK18GYnCtNn8U_b7hhNTFgf3gJzXUEOtrn8L_uuthw7xG5LbY9Oro9myJvJKd2bZ84weNcA4EgTLVJ45AFVin1GecDXOkbJgXKf6pvqpQbBy_Fe8gxbM_A3ZJT6JrXKURhLempoczh7uTTM25y9ybmBJqnm7YjLTGRMy7DwoNIY6oZnVHWY_i04y5MPCDcGOVhn9bALuQ6s1XKZ6AsjEKbT3ytRKd2Q0LZnr7RSkfbk4LNYPtjKgQsVclC8Gao4t6-R76JebQn3GH65RNJr3zxogrDJk4OglgcaeasAou45tdfpBEj4l30DyYWUXcYqxNfsQxDlPaLGqEg30XR9bwKc8Kr57RFZ_16v5SccL6vYQoPO2g&refId=eGb0QUUekOjgqRmAiPqWmQ%3D%3D&trackingId=WfffkrTIdiq6gzEV2lzO5w%3D%3D&trk=flagship3_search_srp_jobs",
    )
)
url_job_postings

,job_title,company,job_desc,location
0,Machine Learning Engineer,Ampstek,"Hi Professionals,Greetings From Ampstek!!Hope ...",Poland
1,Machine Learning Consultant,Antdata,Antdata is a data analytics company that focus...,"Poznań, Wielkopolskie, Poland"
2,Senior Data Scientist with Python and ML,Luxoft Poland,Project descriptionSuccessful candidate will b...,Poland
3,Senior Artificial Intelligence Engineer,Tangent,"About TangentAt Tangent, we partner with large...","Poznań, Wielkopolskie, Poland"
4,Machine Learning Researcher,RTB House,RTB House owes its market position to an advan...,"Warsaw, Mazowieckie, Poland"
5,AI Engineer,Opera,"Location: Wrocław, PolandAbout the job We are ...","Wrocław, Dolnośląskie, Poland"
6,AI Developer with Python,Rockwell Automation,Rockwell Automation is a global technology lea...,"Katowice, Śląskie, Poland"
7,Machine Learning Engineer,Devoteam G Cloud,"Company DescriptionDevoteam, a group of 8000+ ...","Warsaw, Mazowieckie, Poland"
8,Generative AI Engineer,Seargin,As a dynamic multinational tech company operat...,Poland
9,AIMl GenAI Manager,EY,AIML GenAI ManagerLet us introduce you the job...,"Wrocław, Dolnośląskie, Poland"


In [3]:
query_job_postings = linkedin_scraper.scrap_queries(
    [Query("ML Engineer", options=QueryOptions(limit=15, locations=("Poland")))]
)
query_job_postings

("AnonymousStrategy is no longer maintained and it won't probably work. It is recommended to use an authenticated session, see documentation at https://github.com/spinlud/py-linkedin-jobs-scraper#anonymous-vs-authenticated-session.",)
('[AnonymousStrategy]', 'This run strategy is no longer supported')


1: Machine Learning Engineer @ Text
2: Machine Learning Engineer - Part Time [Permanent recruitment] @ AI REV
3: Senior AI/ML Engineer @ Alpheya
4: Machine Learning Engineer @ Devoteam G Cloud
5: AI/ML Engineer @ gravity9
6: Middle/Senior ML Engineer @ Provectus
7: Machine Learning Engineer @ HackerPulse
8: Machine Learning Engineer @ Vertex Agility
9: Machine Learning Engineer @ Ampstek
10: ML Engineer @ NeuReality
11: Machine Learning Engineer @ Allegro
12: Artificial Intelligence Engineer for pre-sale activities @ Kitrum
13: Machine Learning Engineer (AI) @ Medius
14: Machine Learning Engineer @ Ampstek
15: Machine Learning Engineer (100% remote-friendly within Poland) @ Docplanner


,job_title,company,job_desc,location
0,Machine Learning Engineer,Text,Ready to dive into exciting machine learning p...,Cracow Metropolitan Area
1,Machine Learning Engineer - Part Time [Permane...,AI REV,AI REV is a boutique partnership of practical ...,"Warsaw, Mazowieckie, Poland"
2,Senior AI/ML Engineer,Alpheya,Description\n\nAbout Alpheya\n\nWe are a B2B W...,"Warsaw, Mazowieckie, Poland"
3,Machine Learning Engineer,Devoteam G Cloud,"\nCompany Description\n\nDevoteam, a group of ...","Warsaw, Mazowieckie, Poland"
4,AI/ML Engineer,gravity9,Join Our Growing Team - Future Opportunities A...,Poland
5,Middle/Senior ML Engineer,Provectus,Join us at Provectus to be a part of a team th...,Cracow Metropolitan Area
6,Machine Learning Engineer,HackerPulse,Showcase your software engineering talents usi...,Poland
7,Machine Learning Engineer,Vertex Agility,Role: Machine Learning Engineer\n\n\n\n\nOur c...,"Cracow, Małopolskie, Poland"
8,Machine Learning Engineer,Ampstek,"Hi Professionals,\n\n\n\n\nGreetings From Amps...",Poland
9,ML Engineer,NeuReality,We are looking for an experienced AI/ML engine...,"Cracow, Małopolskie, Poland"


CVs have to be provided as plain text.

The desired _job_title_ will be generated from the CV unless provided explicitely to the _get_missing_skills_ method. It is not viable to prompt each job for _key_points_, so _job_title_ is used to filter out only 10 the most relevant postings.


In [6]:
cv_1 = None
with open("data/cv_1.txt", "r") as file:
    cv_1 = file.read()

cv_2 = None
with open("data/cv_3.txt", "r") as file:
    cv_2 = file.read()

cv_3 = None
with open("data/cv_3.txt", "r") as file:
    cv_3 = file.read()

## Exemplary usage


In [7]:
cover_letter = job_market_assistant.get_cover_letter(url_job_postings.iloc[0], cv_1)

### Cover letter for Applied Data Scientist at dunnhumby:
[Your Name]
[Address]
[City, State ZIP Code]
[Email Address]
[Today’s Date]

[Employer’s Name]
[Company Name]
[Address]
[City, State ZIP Code]

Dear Hiring Manager,

I am excited to apply for the Applied Data Scientist position at dunnhumby, as advertised on [where you found the job posting]. With my educational background in Artificial Intelligence and Finance, combined with my experience as a C++ Developer and Cook, I believe I am a strong candidate for this role.

I am currently in my final year of Artificial Intelligence and have taken various courses such as Data Visualization, NLP, ML, DL, BigData, Information Retrieval, Computer Vision, and Data Mining. These courses have equipped me with the necessary skills to become an Applied Data Scientist. Additionally, I am a member of GHOST, an academic circle focused on AI/ML, and I am currently part of the section organizing the GHOST conference about ML.

In my current role as a C++ Developer at CAD Projekt, I have gained experience in programming languages such as Python, C++, SQL, R, and Java, which are required for this position. I have also developed strong analytical techniques and a passion for connecting my work directly to the customer experience, making a real and tangible impact. My logical thinking and problem-solving skills, along with excellent communication skills, have enabled me to successfully work on complex applications and components in a team environment.

One of the things that caught my attention in the job posting was the company's focus on diversity and inclusion. I strongly believe in creating an inclusive culture where everyone has equal opportunities to thrive, and I am eager to contribute to this at dunnhumby.

As a person who values a work-life balance, I appreciate the company's approach to flexible working hours and thoughtful perks such as flexible working hours and your birthday off. I feel that this will create an environment where employees can perform their best and contribute to the company's success.

I am excited about the opportunity to work with cutting-edge technology at dunnhumby, and I am confident that my skills and experience make me a strong fit for this role. I am eager to learn and contribute to the company's mission of enabling businesses to grow and reimagine themselves by becoming advocates and champions for their Customers.

Thank you for considering my application. I have attached my resume for your review, and I look forward to the opportunity to discuss further my qualifications and how I can contribute to dunnhumby. Please

In [8]:
missing_skills = job_market_assistant.get_missing_skills(query_job_postings, cv_2)

/home/vrepetskyi/classes/6/nlp/job-market-assistant/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Postings best matching job title Computer Engineer and Data Science Student (derived from the CV)

,job_title,company,job_desc,location,score
0,Graduate Data Engineer,Oho Group Ltd,Graduate Data Engineer - Fast Growing FinTech\...,"London Area, United Kingdom",0.728353
1,"Data Scientist, People & Culture",Lifelancer,We are ideally looking for a candidate in the ...,"London, England, United Kingdom",0.717228
2,Data Scientist,Tilt,"Mission\n\nAt TILT, we’re pioneering the futur...","London, England, United Kingdom",0.712310
3,Data Engineer,ADLIB Recruitment | B Corp™,"Be part of a unique team, bridging commercial ...","Bristol, England, United Kingdom",0.712009
4,Data Engineer,Adria Solutions Ltd,We are seeking a Data Engineer to join our fri...,"Manchester, England, United Kingdom",0.711317
5,Machine Learning Engineer,myGwork - LGBTQ+ Business Community,This inclusive employer is a member of myGwork...,"London, England, United Kingdom",0.709216
6,Graduate Machine Learning Engineer,Mercedes-AMG PETRONAS Formula One Team,"Brackley, UK\n\nWe have an exciting opening fo...","Brackley, England, United Kingdom",0.702707
7,Analytics Engineer,Mars,Job Description\n\nThe Analytics Engineer at M...,"London, England, United Kingdom",0.702397
8,Data Scientist ( Hybrid),myGwork - LGBTQ+ Business Community,This inclusive employer is a member of myGwork...,"Dublin, County Dublin, Ireland",0.700871
9,Data Engineer,ecocareers,DATA ENGINEER LONDON HYBRID - 3 DAYS IN THE OF...,"London, England, United Kingdom",0.698267


### Missing skills:
1. Experience with cloud computing platforms (e.g. AWS, Google Cloud Platform)
2. Containerization and orchestration (e.g. Docker, Kubernetes)
3. Web development (e.g. JavaScript, HTML, CSS)
4. Data visualization (e.g. Tableau, Power BI)
5. Database design and management
6. Agile development
7. Version control systems (e.g. Git)
8. Natural language processing (NLP)
9. Deep learning frameworks (e.g. TensorFlow, PyTorch)
10. Software engineering principles/best practices

In [11]:
missing_skills = job_market_assistant.get_missing_skills(
    file_job_postings, cv_3, "Azure Data Mining Engineer"
)

/home/vrepetskyi/classes/6/nlp/job-market-assistant/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Postings best matching job title Azure Data Mining Engineer

,job_title,company,job_desc,location,score
0,Machine Learning Engineer,myGwork - LGBTQ+ Business Community,This inclusive employer is a member of myGwork...,"London, England, United Kingdom",0.775601
1,Data Engineer,ecocareers,DATA ENGINEER LONDON HYBRID - 3 DAYS IN THE OF...,"London, England, United Kingdom",0.769516
2,Data Engineer,Adria Solutions Ltd,We are seeking a Data Engineer to join our fri...,"Manchester, England, United Kingdom",0.756505
3,Machine Learning Engineer,Stackfix,"About Us\n\nAt Stackfix, we’re transforming ho...","London, England, United Kingdom",0.745697
4,Machine Learning Engineer,Connex One,Join us in pushing the boundaries in Conversat...,"Manchester Area, United Kingdom",0.742178
5,Artificial Intelligence Engineer,Bondex,About The Company\n\nWe are an early-stage (Se...,Greater Portsmouth Area,0.740576
6,"Machine Learning Engineer - UK, Hybrid",Multiverse,We’re on a mission to provide equitable access...,"London, England, United Kingdom",0.738025


### Missing skills:
1. Experience with cloud computing platforms such as AWS or Google Cloud
2. Familiarity with database management systems like MySQL or MongoDB
3. Proficiency in HTML, CSS, and JavaScript
4. Knowledge of software development methodologies, such as Agile or Scrum
5. Experience with data visualization tools like Tableau or Power BI
6. Familiarity with DevOps tools and practices
7. Understanding of Natural Language Processing (NLP) concepts and techniques
8. Knowledge of front-end frameworks such as React or Angular
9. Experience with version control systems like Git
10. Understanding of data warehousing concepts and data modeling

## Benchmarks


### Experimenting with prompts


In [8]:
dream_job1 = "AI Engineer intern working on NLP problems without using some Internet Applications languages"
dream_job2 = "I would like to work in fast developing start-up working on Computer Vision problems"
dream_job3 = "I student without experience, which dream job is AI Software Engineer role at a leading coorporation"

In [10]:
job_market_assistant = JobMarketAssistant(
    model=chat_gpt,
    cover_letter_prompt="Write a cover letter for this job. Do not use too long and unnecessarily sophisticated words.\n\nMy CV:\n{cv}\n\nJob Posting:\n{job_desc}",
    job_title_prompt="Summarize the job title based on the following CV in maximum 10 words:\n\n{cv}\n\nProvide only the job title as plain text without any quotes and punctuation.",
    key_points_prompt="Job Posting:\n\n{job_desc}\n\nGenerate key points from these job posting (focus on skils and requirements) sorted from the most important to the least",
    missing_skills_prompt="This is my CV:\n\n{cv}\n\nReturn me list of skills that I should learn and I don't have in my CV to become more employable for jobs with requirements listed below sorted from the most important to the least. The list has to be in Markdown format. Job postings:\n\n",
)

cover_letter = job_market_assistant.get_cover_letter(file_job_postings.iloc[0], cv_1)
missing_skills = job_market_assistant.get_missing_skills(file_job_postings, cv_1, dream_job1)

### Cover letter for Applied Data Scientist at dunnhumby:
Dear Hiring Manager,

I am writing to express my interest in the Applied Data Scientist position at dunnhumby. With a passion for utilizing my skills in artificial intelligence, programming and data analysis, I am confident that I have the necessary experience and knowledge to contribute to your thriving organization.

My education background includes a focus on Automatics and Robotics, Finance and Accounting, and Artificial Intelligence. I have completed multiple courses in Data Visualization, NLP, ML, DL, BigData, Information Retrieval, Computer Vision and Data Mining, which have strengthened my expertise in this field. I am also a proud member of the academic circle GHOST, where I have been actively involved in organizing the GHOST conference on ML.

In my previous role as a C++ Developer at CAD Projekt, I have gained valuable experience in applying my knowledge of programming languages such as Python, C++, SQL, R, and Java. Additionally, my employment history also includes roles in various industries, such as being a cook at Prezzo and a factory worker at MSA Manufacturing, where I have developed strong analytical thinking and problem-solving skills.

What sets me apart from other candidates is my ability to connect my work directly to the customer experience, resulting in tangible and impactful solutions for clients. I am excited about the opportunity to join your advanced data science team and utilize my skills to distill complex problems into compelling insights.

Working at dunnhumby would not only mean being a part of a global leader in Customer Data Science but also having the chance to work alongside super-smart colleagues who are redefining the rules in this industry. I am confident that my experience and passion align with your mission to enable businesses to be Customer First.

Thank you for considering my application. I look forward to discussing how I can contribute to your organization in more detail.

Sincerely,
[Your Name]

/home/krzychu/anaconda3/envs/nlp_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Postings best matching job title AI Engineer intern working on NLP problems without using some Internet Applications languages

,job_title,company,job_desc,location,score
0,Machine Learning Engineer,Connex One,Join us in pushing the boundaries in Conversat...,"Manchester Area, United Kingdom",0.745181
1,Machine Learning Engineer,Speechmatics,Speechmatics is a cutting-edge applied AI Rese...,"Cambridge, England, United Kingdom",0.742900
2,Machine Learning Engineer,Speechmatics,We aim to get you onboarded and started on som...,"Cambridge, England, United Kingdom",0.730420
3,Machine Learning Engineer,Stackfix,"About Us\n\nAt Stackfix, we’re transforming ho...","London, England, United Kingdom",0.720131
4,Artificial Intelligence Engineer,Bondex,About The Company\n\nWe are an early-stage (Se...,Greater Portsmouth Area,0.714325
5,Machine Learning Engineer - ML Ops,Builder.ai,Description\n\nAbout Builder.ai\n\nWe’re on a ...,"London, England, United Kingdom",0.707100
6,AI Integrations Engineer (Remote),Telnyx,About Telnyx\n\nTelnyx is an industry leader t...,"Dublin, County Dublin, Ireland",0.704437


### Missing skills:
1. Programming languages: Scala, JavaScript, Swift
2. Cloud computing: AWS, Azure, Google Cloud Platform
3. Web development: HTML, CSS, JavaScript frameworks (React, Angular, Vue.js)
4. Agile methodology and project management
5. Mobile app development: iOS and Android platforms
6. DevOps tools: Docker, Kubernetes, Jenkins
7. Data analysis and visualization tools: Tableau, Power BI, Google Analytics
8. Database management: MongoDB, MySQL, Oracle
9. Natural language processing (NLP)
10. Machine learning frameworks: TensorFlow, Keras, PyTorch
11. Blockchain technology
12. Internet of Things (IoT) development
13. Cybersecurity and ethical hacking
14. Virtual and augmented reality development
15. UX/UI design
16. Social media marketing
17. Digital marketing strategies
18. Business intelligence and data warehousing.

In [12]:
job_market_assistant = JobMarketAssistant(
    model=chat_gpt,
    cover_letter_prompt="Write a cover letter for this job. Do not use too long and unnecessarily sophisticated words.\n\nMy CV:\n{cv}\n\nJob Posting:\n{job_desc}",
    job_title_prompt="Summarize the job title based on the following CV in maximum 10 words:\n\n{cv}\n\nProvide only the job title as plain text without any quotes and punctuation.",
    key_points_prompt="Job Posting:\n\n{job_desc}\n\nGenerate key points from these job posting (focus on skils and requirements) sorted from the most important to the least",
    missing_skills_prompt="This is my CV:\n\n{cv}\n\nReturn me list of skills that I should learn and I don't have in my CV to become more employable for jobs with requirements listed below sorted from the most important to the least. The list has to be in Markdown format. Job postings:\n\n",
)

cover_letter = job_market_assistant.get_cover_letter(file_job_postings.iloc[0], cv_2)
missing_skills = job_market_assistant.get_missing_skills(file_job_postings, cv_2, dream_job2)

### Cover letter for Applied Data Scientist at dunnhumby:
Dear Hiring Manager,

I am writing to express my interest in the Applied Data Scientist position at dunnhumby. With a Bachelor's degree in Computer Engineering and currently pursuing a Master's in Data Science, I am confident that my education and skills align perfectly with the requirements of this role. I am excited about the opportunity to join a leading company in the field of Customer Data Science and to contribute my expertise towards enabling businesses to become Customer First.

As seen in my attached resume, I have a strong educational background in both computer engineering and data science, with a focus on hardware and systems as well as courses in statistics, data mining, machine learning, big data, and information retrieval. I have also gained practical experience through my internship at Microsoft during the summer of 2021, where I worked on projects involving data analysis and visualization. As a member of the academic club, Data Science Society, I am currently part of the team organizing the annual Data Science conference, where I have honed my skills in communication, teamwork, and problem-solving.

My proficiency in programming languages such as Python, R, SQL, and C# coupled with my experience in Microsoft Azure will allow me to make valuable contributions to your team. I also possess strong analytical and problem-solving skills, as well as a passion for connecting data to customer experiences and making a tangible impact. I am confident that my skills and experience make me a strong candidate for the Applied Data Scientist role at dunnhumby.

Furthermore, I appreciate dunnhumby's commitment to diversity and inclusive culture, as shown through its various employee networks such as dh Gender Equality Network, dh Proud, dh Family, dh One, and dh Thrive. I am personally dedicated to the value of diversity and believe that inclusion leads to innovation and better problem-solving. I am excited about the opportunity to work in such an environment.

If given the chance, I am confident that I can exceed your expectations and contribute positively to dunnhumby's mission to enable businesses to grow and reimagine themselves while becoming advocates for their customers. I am excited about the possibility of joining your team and would be grateful for the opportunity to further discuss my qualifications and how I can contribute to dunnhumby.

Thank you for considering my application. I look forward to the opportunity of meeting with you to discuss my candidacy further.

Sincerely,
[Your Name]

/home/krzychu/anaconda3/envs/nlp_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Postings best matching job title I would like to work in fast developing start-up working on Computer Vision problems

,job_title,company,job_desc,location,score
0,Graduate Machine Vision Engineer,Bytronic Vision Intelligence,Graduate - Vision Engineer\n\nLocation: Bromsg...,"Blackwell, England, United Kingdom",0.756533
1,IQ Function Owner,Valeo,"Valeo is a tech global company, designing brea...","Tuam, County Galway, Ireland",0.719615
2,Software Engineer (C++),Deep Render,Deep Render is a Deep Tech startup founded to ...,"London, England, United Kingdom",0.712308
3,C++ Developer (f/m/d) in Berlin and REMOTE 130...,think-cell Software,Algorithms\n\nthink-cell was founded on the id...,"Dublin, County Dublin, Ireland",0.704088
4,Machine Learning Engineer,myGwork - LGBTQ+ Business Community,This inclusive employer is a member of myGwork...,"London, England, United Kingdom",0.698591
5,Data Scientist ( Hybrid),myGwork - LGBTQ+ Business Community,This inclusive employer is a member of myGwork...,"Dublin, County Dublin, Ireland",0.698440
6,AI Software Engineer (C++/GPGPU),microTECH Global LTD,The ideal candidate will have experience and e...,"Egham, England, United Kingdom",0.689220
7,Software Development Engineer,myGwork - LGBTQ+ Business Community,Experience programming with at least one softw...,"Dublin, County Dublin, Ireland",0.687555


### Missing skills:
**1. Experience with Statistical Software**
- SPSS
- STATA
- SAS
- MATLAB

**2. Knowledge of Data Visualization Tools**
- Tableau
- PowerBI
- Qlikview
- IBM Cognos

**3. Experience with Relational Database Management Systems**
- Oracle
- MySQL
- PostgreSQL
- IBM DB2

**4. Familiarity with NoSQL Databases**
- MongoDB
- HBase
- Cassandra
- Redis

**5. Understanding of Distributed Computing**
- Apache Spark
- Apache Hadoop
- MapReduce
- Apache Storm

**6. Knowledge of Data Warehousing**
- Teradata
- Amazon Redshift
- Snowflake
- Microsoft Azure Data Warehouse

**7. Experience with Natural Language Processing (NLP)**
- NLTK
- Gensim
- SpaCy
- Stanford NLP

**8. Understanding of Deep Learning Frameworks**
- TensorFlow
- PyTorch
- Keras
- Caffe

**9. Familiarity with Data Governance and Compliance**
- GDPR
- HIPAA
- CCPA
- ISO/IEC 27001

**10. Knowledge of Business Intelligence Tools**
- Microsoft Power BI
- SAP BusinessObjects
- MicroStrategy
- IBM Cognos Analytics

In [13]:
job_market_assistant = JobMarketAssistant(
    model=chat_gpt,
    cover_letter_prompt="Write a cover letter for this job. Do not use too long and unnecessarily sophisticated words.\n\nMy CV:\n{cv}\n\nJob Posting:\n{job_desc}",
    job_title_prompt="Summarize the job title based on the following CV in maximum 10 words:\n\n{cv}\n\nProvide only the job title as plain text without any quotes and punctuation.",
    key_points_prompt="Job Posting:\n\n{job_desc}\n\nGenerate key points from these job posting (focus on skils and requirements) sorted from the most important to the least",
    missing_skills_prompt="This is my CV:\n\n{cv}\n\nReturn me list of skills that I should learn and I don't have in my CV to become more employable for jobs with requirements listed below sorted from the most important to the least. The list has to be in Markdown format. Job postings:\n\n",
)

cover_letter = job_market_assistant.get_cover_letter(file_job_postings.iloc[0], cv_3)
missing_skills = job_market_assistant.get_missing_skills(file_job_postings, cv_3, dream_job3)

### Cover letter for Applied Data Scientist at dunnhumby:
[Your Name]
[Your Address]
[City, State ZIP Code]
[Your Email Address]
[Today’s Date]

[Employer's Name]
[Company Name]
[Company Address]
[City, State ZIP Code]

Dear [Employer's Name],

As a recent graduate with a Bachelor's degree in Computer Engineering and ongoing studies in Data Science, I am excited to express my interest in the Applied Data Scientist position at dunnhumby. With a strong passion for utilizing data to drive business growth, I believe I possess the necessary skills and experience to be a valuable asset to your team.

With a focus on Hardware and Systems in my Computer Engineering degree, I have developed a solid understanding of data structures and algorithms. My ongoing studies in Data Science have equipped me with knowledge in statistics, data mining, machine learning, big data, and information retrieval. I also have hands-on experience in these areas through my internship at Microsoft during the summer of 2021 and my involvement in the Data Science Society, where I am currently a part of the team organizing the annual Data Science conference.

In addition to my academic achievements, I have practical skills in programming languages such as Python, R, SQL, and experience with Microsoft Azure. I also hold a valid driving license and have a C1 proficiency level in English.

The job posting for the Applied Data Scientist role at dunnhumby caught my attention as it aligns perfectly with my interests and goals. I am eager to apply my expertise in machine learning and analytical techniques to solve complex problems and provide impactful insights to clients. The opportunity to work alongside highly skilled colleagues and challenge the status quo is an exciting prospect for me.

Moreover, I am impressed by dunnhumby's focus on putting the customer first and its commitment to diversity and inclusion. I believe that my experience and passion for connecting data to the customer experience make me a strong fit for this role.

I am confident that my logical thinking, problem-solving abilities, and strong communication skills will enable me to excel in this role. I am also familiar with statistical modeling and have experience in applying data science to solve client problems.

Lastly, I appreciate dunnhumby's approach to flexible working and promoting a healthy work-life balance. As a result, I believe that I would be able to deliver my best work while maintaining a fulfilling personal life.

Thank you for considering my application. I am excited about the opportunity to contribute my skills, experience, and passion to dunnhumby's mission. I look forward to discussing my qualifications further

/home/krzychu/anaconda3/envs/nlp_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Postings best matching job title I student without experience, which dream job is AI Software Engineer role at a leading coorporation

,job_title,company,job_desc,location,score
0,AI Software Engineer (C++/GPGPU),Extended Family International,6 Month Contract\n\nThe ideal candidate will h...,"Egham, England, United Kingdom",0.712317
1,AI Software Engineer (C++/GPGPU),microTECH Global LTD,The ideal candidate will have experience and e...,"Egham, England, United Kingdom",0.710474
2,Machine Learning Engineer - ML Ops,Builder.ai,Description\n\nAbout Builder.ai\n\nWe’re on a ...,"London, England, United Kingdom",0.675666
3,Machine Learning Engineer,Stackfix,"About Us\n\nAt Stackfix, we’re transforming ho...","London, England, United Kingdom",0.675324
4,Software Engineer (C++),Deep Render,Deep Render is a Deep Tech startup founded to ...,"London, England, United Kingdom",0.670702
5,Software Engineer (Ireland),HubSpot,Confidence can sometimes hold us back from app...,Ireland,0.663935
6,Software Engineer (Controls - Simulation),Oxa,Description\n\nOxa is enabling the transition ...,"Oxford, England, United Kingdom",0.660449
7,Artificial Intelligence Engineer,Bondex,About The Company\n\nWe are an early-stage (Se...,Greater Portsmouth Area,0.657932


### Missing skills:
1. Experience with cloud computing platforms such as AWS or Google Cloud
2. Knowledge of database management and data warehousing
3. Familiarity with programming languages such as Java or JavaScript
4. Understanding of data visualization and reporting tools such as Tableau or Power BI
5. Experience with project management and Agile methodologies
6. Proficiency in Microsoft Excel and data analysis
7. Knowledge of natural language processing and text mining techniques
8. Understanding of predictive modeling and forecasting methods
9. Familiarity with Hadoop or Spark for distributed computing
10. Knowledge of deep learning and neural networks for advanced data analysis.

#### Gemini

In [14]:
job_market_assistant = JobMarketAssistant(
    model=gemini,
    cover_letter_prompt="Write a cover letter for this job. Do not use too long and unnecessarily sophisticated words.\n\nMy CV:\n{cv}\n\nJob Posting:\n{job_desc}",
    job_title_prompt="Summarize the job title based on the following CV in maximum 10 words:\n\n{cv}\n\nProvide only the job title as plain text without any quotes and punctuation.",
    key_points_prompt="Job Posting:\n\n{job_desc}\n\nGenerate key points from these job posting (focus on skils and requirements) sorted from the most important to the least",
    missing_skills_prompt="This is my CV:\n\n{cv}\n\nReturn me list of skills that I should learn and I don't have in my CV to become more employable for jobs with requirements listed below sorted from the most important to the least. The list has to be in Markdown format. Job postings:\n\n",
)

cover_letter = job_market_assistant.get_cover_letter(file_job_postings.iloc[0], cv_1)
missing_skills = job_market_assistant.get_missing_skills(file_job_postings, cv_1, dream_job1)

### Cover letter for Applied Data Scientist at dunnhumby:
## Dear [Hiring Manager name],

I am writing to express my keen interest in the Applied Data Scientist position at dunnhumby, as advertised on [website name]. With my strong background in Artificial Intelligence, combined with my practical experience in data analysis and programming, I am confident that I possess the skills and qualifications necessary to excel in this role.

**My passion for data science** stems from my desire to utilize data to solve complex problems and drive impactful results. My academic background in Artificial Intelligence, including courses in Data Visualization, NLP, ML, DL, BigData, Information Retrieval, Computer Vision, and Data Mining, has equipped me with a comprehensive understanding of data analysis techniques and algorithms.

**My professional experience** as a C++ Developer at CAD Projekt has further honed my programming skills, particularly in Python, C++, SQL, R, and Java. I have also gained valuable experience in applying data analysis principles to real-world projects, contributing to the development of efficient and effective

/home/krzychu/anaconda3/envs/nlp_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Postings best matching job title AI Engineer intern working on NLP problems without using some Internet Applications languages

,job_title,company,job_desc,location,score
0,Machine Learning Engineer,Connex One,Join us in pushing the boundaries in Conversat...,"Manchester Area, United Kingdom",0.745181
1,Machine Learning Engineer,Speechmatics,Speechmatics is a cutting-edge applied AI Rese...,"Cambridge, England, United Kingdom",0.742900
2,Machine Learning Engineer,Speechmatics,We aim to get you onboarded and started on som...,"Cambridge, England, United Kingdom",0.730420
3,Machine Learning Engineer,Stackfix,"About Us\n\nAt Stackfix, we’re transforming ho...","London, England, United Kingdom",0.720131
4,Artificial Intelligence Engineer,Bondex,About The Company\n\nWe are an early-stage (Se...,Greater Portsmouth Area,0.714325
5,Machine Learning Engineer - ML Ops,Builder.ai,Description\n\nAbout Builder.ai\n\nWe’re on a ...,"London, England, United Kingdom",0.707100
6,AI Integrations Engineer (Remote),Telnyx,About Telnyx\n\nTelnyx is an industry leader t...,"Dublin, County Dublin, Ireland",0.704437


### Missing skills:
## Skills to Learn for Employability

Based on your CV and the provided job postings, here's a list of skills you could learn to improve your employability, sorted from most to least important:

**1. Deep Learning Frameworks:**

* TensorFlow
* PyTorch
* Keras

**2. Cloud Computing Platforms:**

* AWS
* Azure
* GCP

**3. Big Data Technologies:**

* Apache Spark
* Hadoop
* Kafka

**4. Advanced Machine Learning Techniques:**

* Reinforcement Learning
* Generative Adversarial Networks (GANs)
* Bayesian Optimization

**5. Computer Vision Libraries:**

* OpenCV
* TensorFlow Object Detection API
* PyTorch Vision

**6. Natural Language Processing Libraries:**

* NLTK
* spaCy
* Gensim

**7. Data Visualization Libraries:**

* Tableau
* Power BI
* Plotly

**8. DevOps Tools:**

* Git
* Docker

In [16]:
job_market_assistant = JobMarketAssistant(
    model=gemini,
    cover_letter_prompt="Write a cover letter for this job. Do not use too long and unnecessarily sophisticated words.\n\nMy CV:\n{cv}\n\nJob Posting:\n{job_desc}",
    job_title_prompt="Summarize the job title based on the following CV in maximum 10 words:\n\n{cv}\n\nProvide only the job title as plain text without any quotes and punctuation.",
    key_points_prompt="Job Posting:\n\n{job_desc}\n\nGenerate key points from these job posting (focus on skils and requirements) sorted from the most important to the least",
    missing_skills_prompt="This is my CV:\n\n{cv}\n\nReturn me list of skills that I should learn and I don't have in my CV to become more employable for jobs with requirements listed below sorted from the most important to the least. The list has to be in Markdown format. Job postings:\n\n",
)

cover_letter = job_market_assistant.get_cover_letter(file_job_postings.iloc[0], cv_2)
missing_skills = job_market_assistant.get_missing_skills(file_job_postings, cv_2, dream_job2)

### Cover letter for Applied Data Scientist at dunnhumby:
## Dear [Hiring Manager name],

I am writing to express my keen interest in the Applied Data Scientist position at dunnhumby, as advertised on [website name]. With my background in Computer Engineering and current studies in Data Science, I am confident that I possess the necessary skills and experience to excel in this role.

During my studies, I have gained a strong foundation in statistics, data mining, machine learning, big data, and information retrieval. I am proficient in Python, R, SQL, and C#, and have experience using Microsoft Azure. Additionally, my internship at Microsoft during the summer of 2021 provided me with valuable hands-on experience in applying data science techniques to real-world problems.

I am particularly drawn to dunnhumby's mission of empowering businesses to become customer-centric through data science. My passion for connecting data analysis to tangible customer experiences aligns perfectly with your company's values. I am confident that I can contribute to your team

/home/krzychu/anaconda3/envs/nlp_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Postings best matching job title I would like to work in fast developing start-up working on Computer Vision problems

,job_title,company,job_desc,location,score
0,Graduate Machine Vision Engineer,Bytronic Vision Intelligence,Graduate - Vision Engineer\n\nLocation: Bromsg...,"Blackwell, England, United Kingdom",0.756533
1,IQ Function Owner,Valeo,"Valeo is a tech global company, designing brea...","Tuam, County Galway, Ireland",0.719615
2,Software Engineer (C++),Deep Render,Deep Render is a Deep Tech startup founded to ...,"London, England, United Kingdom",0.712308
3,C++ Developer (f/m/d) in Berlin and REMOTE 130...,think-cell Software,Algorithms\n\nthink-cell was founded on the id...,"Dublin, County Dublin, Ireland",0.704088
4,Machine Learning Engineer,myGwork - LGBTQ+ Business Community,This inclusive employer is a member of myGwork...,"London, England, United Kingdom",0.698591
5,Data Scientist ( Hybrid),myGwork - LGBTQ+ Business Community,This inclusive employer is a member of myGwork...,"Dublin, County Dublin, Ireland",0.698440
6,AI Software Engineer (C++/GPGPU),microTECH Global LTD,The ideal candidate will have experience and e...,"Egham, England, United Kingdom",0.689220
7,Software Development Engineer,myGwork - LGBTQ+ Business Community,Experience programming with at least one softw...,"Dublin, County Dublin, Ireland",0.687555


### Missing skills:
## Skills to Learn for Increased Employability

Based on your CV and the provided job postings, here's a list of skills you could learn to become more competitive in the job market:

**High Priority:**

* **Deep Learning:** This is a rapidly growing field with high demand in various industries. Mastering deep learning techniques like convolutional neural networks (CNNs) and recurrent neural networks (RNNs) can significantly boost your employability.
* **Cloud Computing:** Familiarity with cloud platforms like AWS, Azure, or GCP is crucial for many data science roles. Learning how to deploy and manage models on the cloud will make you a more attractive candidate.
* **Data Visualization:** Effectively communicating insights through compelling visualizations is essential for data scientists. Mastering libraries like Tableau, Power BI, or Plotly will enhance your ability to present your findings.
* **Domain Expertise:** Having domain knowledge in the specific industry you're targeting can give you a significant edge. For example, if you

In [28]:
job_market_assistant = JobMarketAssistant(
    model=gemini,
    cover_letter_prompt="Write a cover letter for this job. Do not use too long and unnecessarily sophisticated words.\n\nMy CV:\n{cv}\n\nJob Posting:\n{job_desc}",
    job_title_prompt="Summarize the job title based on the following CV in maximum 10 words:\n\n{cv}\n\nProvide only the job title as plain text without any quotes and punctuation.",
    key_points_prompt="Job Posting:\n\n{job_desc}\n\nGenerate key points from these job posting (focus on skils and requirements) sorted from the most important to the least",
    missing_skills_prompt="This is my CV:\n\n{cv}\n\nReturn me list of skills that I should learn and I don't have in my CV to become more employable for jobs with requirements listed below sorted from the most important to the least. The list has to be in Markdown format. Job postings:\n\n",
)

cover_letter = job_market_assistant.get_cover_letter(file_job_postings.iloc[0], cv_3)
missing_skills = job_market_assistant.get_missing_skills(file_job_postings, cv_3, dream_job3)

### Cover letter for Applied Data Scientist at dunnhumby:
## Dear [Hiring Manager name],

I am writing to express my keen interest in the Applied Data Scientist position at dunnhumby, as advertised on [website]. With my background in computer engineering and data science, combined with my passion for customer-centric solutions, I believe I possess the skills and experience necessary to excel in this role.

My academic background includes a Bachelor's degree in Computer Engineering with a focus on Hardware and Systems, and I am currently pursuing a Master's degree in Data Science, specializing in Statistics, Data Mining, Machine Learning, Big Data, and Information Retrieval. During my studies, I have gained a strong foundation in data analysis, modeling, and visualization techniques, which I have further honed through my internship at Microsoft during the summer of 2021.

My technical skills include proficiency in Python, R, SQL, and C#, as well as experience with Microsoft Azure. I am also a highly motivated and analytical individual with strong problem-solving abilities

/home/krzychu/anaconda3/envs/nlp_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Postings best matching job title I student without experience, which dream job is AI Software Engineer role at a leading coorporation

,job_title,company,job_desc,location,score
0,AI Software Engineer (C++/GPGPU),Extended Family International,6 Month Contract\n\nThe ideal candidate will h...,"Egham, England, United Kingdom",0.712317
1,AI Software Engineer (C++/GPGPU),microTECH Global LTD,The ideal candidate will have experience and e...,"Egham, England, United Kingdom",0.710474
2,Machine Learning Engineer - ML Ops,Builder.ai,Description\n\nAbout Builder.ai\n\nWe’re on a ...,"London, England, United Kingdom",0.675666
3,Machine Learning Engineer,Stackfix,"About Us\n\nAt Stackfix, we’re transforming ho...","London, England, United Kingdom",0.675324
4,Software Engineer (C++),Deep Render,Deep Render is a Deep Tech startup founded to ...,"London, England, United Kingdom",0.670702
5,Software Engineer (Ireland),HubSpot,Confidence can sometimes hold us back from app...,Ireland,0.663935
6,Software Engineer (Controls - Simulation),Oxa,Description\n\nOxa is enabling the transition ...,"Oxford, England, United Kingdom",0.660449
7,Artificial Intelligence Engineer,Bondex,About The Company\n\nWe are an early-stage (Se...,Greater Portsmouth Area,0.657932


### Missing skills:
## Skills to Learn for Employability

Based on your CV and the provided job postings, here's a list of skills you could learn to improve your employability, sorted from most to least important:

**1. Deep Learning:** This is a rapidly growing field with high demand in various industries. Mastering deep learning techniques like convolutional neural networks (CNNs), recurrent neural networks (RNNs), and transformers will significantly boost your value as a data scientist.

**2. Cloud Computing:** Familiarity with cloud platforms like AWS, Azure, or GCP is crucial for modern data science workflows. Learn how to deploy and manage machine learning models, process data, and build scalable data pipelines on the cloud.

**3. Advanced Statistics:** A strong foundation in statistical concepts like hypothesis testing, Bayesian inference, and time series analysis is essential for rigorous data analysis and model evaluation.

**4. Natural Language Processing (NLP):** With the increasing importance of text data, NLP skills are becoming highly sought

#### Llama

In [20]:
job_market_assistant = JobMarketAssistant(
    model=llama,
    cover_letter_prompt="Write a cover letter for this job. Do not use too long and unnecessarily sophisticated words.\n\nMy CV:\n{cv}\n\nJob Posting:\n{job_desc}",
    job_title_prompt="Summarize the job title based on the following CV in maximum 10 words:\n\n{cv}\n\nProvide only the job title as plain text without any quotes and punctuation.",
    key_points_prompt="Job Posting:\n\n{job_desc}\n\nGenerate key points from these job posting (focus on skils and requirements) sorted from the most important to the least",
    missing_skills_prompt="This is my CV:\n\n{cv}\n\nReturn me list of skills that I should learn and I don't have in my CV to become more employable for jobs with requirements listed below sorted from the most important to the least. The list has to be in Markdown format. Job postings:\n\n",
)

cover_letter = job_market_assistant.get_cover_letter(file_job_postings.iloc[0], cv_1)
missing_skills = job_market_assistant.get_missing_skills(file_job_postings, cv_1, dream_job1)

### Cover letter for Applied Data Scientist at dunnhumby:
Here is a cover letter tailored to the job posting:

Dear Hiring Manager,

I am excited to apply for the Applied Data Scientist position at dunnhumby. With a strong background in Artificial Intelligence and a passion for using data to drive business decisions, I am confident that my skills and experience make me an ideal candidate for this role.

As a current student of Artificial Intelligence, I have developed a solid foundation in programming languages such as Python, C++, SQL, and R. My experience as a C++ Developer at CAD Projekt has given me hands-on experience with software development and problem-solving. Additionally, my academic background in Automatics and Robotics, Finance and Accounting, and Artificial Intelligence has provided me with a strong understanding of data analysis and visualization.

I am particularly drawn to dunnhumby's mission to empower businesses to compete and thrive in the modern data-driven economy. As someone who is passionate about using data to drive business decisions, I am excited about the opportunity to join a team that is dedicated to putting the customer first.

In my current role as a C++ Developer, I have developed strong analytical and problem-solving skills, which I believe would be valuable in this role. I am also confident in my ability to communicate complex ideas effectively and work collaboratively as part of a team.

I am impressed by dunnhumby's commitment to diversity and inclusion and its focus on creating an inclusive culture. As someone who values flexibility and work-life balance, I appreciate the company's approach to flexible working and its commitment to supporting employees' personal and professional goals.

Thank you for considering my application. I would welcome the opportunity to discuss my qualifications further and learn more about the team's work.

Sincerely,
[Your Name]

/home/krzychu/anaconda3/envs/nlp_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Postings best matching job title AI Engineer intern working on NLP problems without using some Internet Applications languages

,job_title,company,job_desc,location,score
0,Machine Learning Engineer,Connex One,Join us in pushing the boundaries in Conversat...,"Manchester Area, United Kingdom",0.745181
1,Machine Learning Engineer,Speechmatics,Speechmatics is a cutting-edge applied AI Rese...,"Cambridge, England, United Kingdom",0.742900
2,Machine Learning Engineer,Speechmatics,We aim to get you onboarded and started on som...,"Cambridge, England, United Kingdom",0.730420
3,Machine Learning Engineer,Stackfix,"About Us\n\nAt Stackfix, we’re transforming ho...","London, England, United Kingdom",0.720131
4,Artificial Intelligence Engineer,Bondex,About The Company\n\nWe are an early-stage (Se...,Greater Portsmouth Area,0.714325
5,Machine Learning Engineer - ML Ops,Builder.ai,Description\n\nAbout Builder.ai\n\nWe’re on a ...,"London, England, United Kingdom",0.707100
6,AI Integrations Engineer (Remote),Telnyx,About Telnyx\n\nTelnyx is an industry leader t...,"Dublin, County Dublin, Ireland",0.704437


### Missing skills:
I'd be happy to help! Unfortunately, I don't see any specific job postings from you. However, I can provide a list of skills that are commonly required in C++ Developer, AI/ML, and related fields, sorted from most important to least important. Here it is:

### Programming languages

* JavaScript (considering your strong background in C++, this will be beneficial for web development and AI/ML applications)
* Swift (for mobile app development and machine learning)
* Julia (for numerical computing and data analysis)

### AI/ML frameworks

* TensorFlow (popular deep learning framework)
* PyTorch (another popular deep learning framework)
* Keras (high-level neural networks API)
* OpenCV (computer vision and image processing library)

### Data analysis and science

* Tableau (data visualization tool)
* Power BI (business analytics service)
* pandas (Python data analysis library)
* scikit-learn (machine learning library for Python)

### Cloud platforms

* AWS (Amazon Web Services) certification
* Azure (Microsoft cloud platform) certification
* Google Cloud Platform (GCP) certification

### Other

* Git (version control system) certification
* Docker (containerization platform) certification
* Data storytelling and presentation skills
* Domain-specific knowledge (e.g., finance, healthcare, etc.)

Please note that this is not an exhaustive list, and the importance of each skill may vary depending on the specific job requirements. Focus on the ones that resonate with your interests and goals.

In [22]:
job_market_assistant = JobMarketAssistant(
    model=llama,
    cover_letter_prompt="Write a cover letter for this job. Do not use too long and unnecessarily sophisticated words.\n\nMy CV:\n{cv}\n\nJob Posting:\n{job_desc}",
    job_title_prompt="Summarize the job title based on the following CV in maximum 10 words:\n\n{cv}\n\nProvide only the job title as plain text without any quotes and punctuation.",
    key_points_prompt="Job Posting:\n\n{job_desc}\n\nGenerate key points from these job posting (focus on skils and requirements) sorted from the most important to the least",
    missing_skills_prompt="This is my CV:\n\n{cv}\n\nReturn me list of skills that I should learn and I don't have in my CV to become more employable for jobs with requirements listed below sorted from the most important to the least. The list has to be in Markdown format. Job postings:\n\n",
)

cover_letter = job_market_assistant.get_cover_letter(file_job_postings.iloc[0], cv_2)
missing_skills = job_market_assistant.get_missing_skills(file_job_postings, cv_2, dream_job2)

### Cover letter for Applied Data Scientist at dunnhumby:
Dear Hiring Manager,

I am excited to apply for the Applied Data Scientist position at dunnhumby, where I can utilize my skills and knowledge to drive business growth and deliver impactful solutions. With a strong foundation in Computer Engineering and Data Science, I am confident in my ability to distill complex problems into compelling insights using machine learning and human creativity.

Throughout my education, I have developed a solid understanding of programming languages such as Python, R, SQL, and C#. My proficiency in Microsoft Azure and experience with internships, such as my summer internship at Microsoft, have further honed my skills. Furthermore, my membership in the Data Science Society has provided me with valuable experience in organizing events, such as the annual Data Science conference.

As a detail-oriented individual with strong analytical skills, I am well-equipped to investigate, develop, implement, and deploy complex applications and components. My logical thinking and problem-solving abilities allow me to communicate complex ideas effectively. Additionally, my experience with statistical modeling and applying data science to client problems has prepared me to make a real and tangible impact.

I am particularly drawn to dunnhumby's mission to empower businesses to compete and thrive in the modern data-driven economy by becoming advocates and champions for their customers. I am excited about the opportunity to work with a global leader in Customer Data Science and apply my expertise to deliver effective and impactful solutions for clients.

In line with the company's commitment to flexibility and inclusivity, I appreciate the emphasis on work-life balance and the opportunity to discuss agile working options during the hiring process. I am confident that my skills, experience, and passion for connecting my work to the customer experience make me a strong fit for this role.

Thank you for considering my application. I look forward to the opportunity to discuss my qualifications further.

Sincerely,
[Your Name]

/home/krzychu/anaconda3/envs/nlp_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Postings best matching job title I would like to work in fast developing start-up working on Computer Vision problems

,job_title,company,job_desc,location,score
0,Graduate Machine Vision Engineer,Bytronic Vision Intelligence,Graduate - Vision Engineer\n\nLocation: Bromsg...,"Blackwell, England, United Kingdom",0.756533
1,IQ Function Owner,Valeo,"Valeo is a tech global company, designing brea...","Tuam, County Galway, Ireland",0.719615
2,Software Engineer (C++),Deep Render,Deep Render is a Deep Tech startup founded to ...,"London, England, United Kingdom",0.712308
3,C++ Developer (f/m/d) in Berlin and REMOTE 130...,think-cell Software,Algorithms\n\nthink-cell was founded on the id...,"Dublin, County Dublin, Ireland",0.704088
4,Machine Learning Engineer,myGwork - LGBTQ+ Business Community,This inclusive employer is a member of myGwork...,"London, England, United Kingdom",0.698591
5,Data Scientist ( Hybrid),myGwork - LGBTQ+ Business Community,This inclusive employer is a member of myGwork...,"Dublin, County Dublin, Ireland",0.698440
6,AI Software Engineer (C++/GPGPU),microTECH Global LTD,The ideal candidate will have experience and e...,"Egham, England, United Kingdom",0.689220
7,Software Development Engineer,myGwork - LGBTQ+ Business Community,Experience programming with at least one softw...,"Dublin, County Dublin, Ireland",0.687555


### Missing skills:
Based on the job postings, I've identified the skills you may want to consider learning to become more employable. Here's the list in Markdown format, sorted from most important to least important:

### Top Priority

* **Cloud Computing**: Familiarity with cloud platforms like AWS, Google Cloud, or Microsoft Azure (you already have experience with Microsoft Azure)
* **Data Visualization**: Tools like Tableau, Power BI, or D3.js to effectively communicate insights and results
* **Machine Learning**: Familiarity with popular libraries like TensorFlow, PyTorch, or Scikit-learn to work with complex data sets

### High Priority

* **Big Data Processing**: Knowledge of Hadoop, Spark, or NoSQL databases like Cassandra or MongoDB
* **Cybersecurity**: Understanding of security principles and practices to ensure data integrity and confidentiality
* **Agile Methodologies**: Familiarity with Agile development methodologies like Scrum or Kanban

### Medium Priority

* **Web Development**: Basic understanding of HTML, CSS, and JavaScript to work with web-based applications
* **Database Management**: Knowledge of database design, normalization, and querying (you already have experience with SQL)
* **Business Intelligence**: Familiarity with business intelligence tools like Power BI, QlikView, or SAP BusinessObjects

### Low Priority

* **Artificial Intelligence**: Familiarity with AI frameworks like OpenCV or TensorFlow Lite for edge computing
* **Internet of Things**: Understanding of IoT concepts and protocols like MQTT or CoAP

Remember to prioritize skills based on your interests, career goals, and the job market. Focus on the most important skills first and gradually move to the lower-priority ones.

In [21]:
job_market_assistant = JobMarketAssistant(
    model=llama,
    cover_letter_prompt="Write a cover letter for this job. Do not use too long and unnecessarily sophisticated words.\n\nMy CV:\n{cv}\n\nJob Posting:\n{job_desc}",
    job_title_prompt="Summarize the job title based on the following CV in maximum 10 words:\n\n{cv}\n\nProvide only the job title as plain text without any quotes and punctuation.",
    key_points_prompt="Job Posting:\n\n{job_desc}\n\nGenerate key points from these job posting (focus on skils and requirements) sorted from the most important to the least",
    missing_skills_prompt="This is my CV:\n\n{cv}\n\nReturn me list of skills that I should learn and I don't have in my CV to become more employable for jobs with requirements listed below sorted from the most important to the least. The list has to be in Markdown format. Job postings:\n\n",
)

cover_letter = job_market_assistant.get_cover_letter(file_job_postings.iloc[0], cv_3)
missing_skills = job_market_assistant.get_missing_skills(file_job_postings, cv_3, dream_job3)

### Cover letter for Applied Data Scientist at dunnhumby:
[Your Name]
[Your Address]
[City, Country]
[Email Address]
[Phone Number]
[Date]

Stephanie Winson
dunnhumby
[Address]
[City, Country]

Dear Stephanie,

I am excited to apply for the Applied Data Scientist position at dunnhumby, as advertised on [Job Posting Website]. With a strong background in computer engineering and data science, I am confident that my skills and experience make me an ideal candidate for this role.

As a recent graduate in Computer Engineering with a focus on Hardware and Systems, I have developed a solid foundation in programming languages such as Python, R, SQL, and C#. My internship at Microsoft during the summer of 2021 has given me hands-on experience with Microsoft Azure. Additionally, I am currently pursuing a degree in Data Science, with a focus on Statistics, Data Mining, Machine Learning, Big Data, and Information Retrieval.

As a member of the Data Science Society, I have been actively involved in organizing the annual Data Science conference. This experience has not only honed my organizational skills but also given me the opportunity to work with a team to achieve a common goal.

I am particularly drawn to dunnhumby's mission to enable businesses to grow and reimagine themselves by becoming advocates and champions for their customers. As someone who is passionate about using data to drive business decisions, I am excited about the opportunity to join a team that is dedicated to putting the customer first.

In addition to my technical skills, I possess excellent communication skills, which I believe are essential for effective collaboration and communication with colleagues and clients. My proficiency in English (C1 level) has also enabled me to effectively communicate with international teams and clients.

I am confident that my unique blend of technical and soft skills makes me an excellent fit for this role. I am excited about the opportunity to join dunnhumby's advanced data science team and contribute to the company's mission.

Thank you for considering my application. I would welcome the opportunity to discuss my application and how I can contribute to the success of dunnhumby.

Sincerely,

[Your Name]

/home/krzychu/anaconda3/envs/nlp_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Postings best matching job title I student without experience, which dream job is AI Software Engineer role at a leading coorporation

,job_title,company,job_desc,location,score
0,AI Software Engineer (C++/GPGPU),Extended Family International,6 Month Contract\n\nThe ideal candidate will h...,"Egham, England, United Kingdom",0.712317
1,AI Software Engineer (C++/GPGPU),microTECH Global LTD,The ideal candidate will have experience and e...,"Egham, England, United Kingdom",0.710474
2,Machine Learning Engineer - ML Ops,Builder.ai,Description\n\nAbout Builder.ai\n\nWe’re on a ...,"London, England, United Kingdom",0.675666
3,Machine Learning Engineer,Stackfix,"About Us\n\nAt Stackfix, we’re transforming ho...","London, England, United Kingdom",0.675324
4,Software Engineer (C++),Deep Render,Deep Render is a Deep Tech startup founded to ...,"London, England, United Kingdom",0.670702
5,Software Engineer (Ireland),HubSpot,Confidence can sometimes hold us back from app...,Ireland,0.663935
6,Software Engineer (Controls - Simulation),Oxa,Description\n\nOxa is enabling the transition ...,"Oxford, England, United Kingdom",0.660449
7,Artificial Intelligence Engineer,Bondex,About The Company\n\nWe are an early-stage (Se...,Greater Portsmouth Area,0.657932


### Missing skills:
Based on the job postings, I've identified the skills you may need to focus on to become more employable. Please find the list below:

### Top Priority Skills
### Cloud Computing
* AWS (Amazon Web Services) certifications or proficiency
* Cloud architecture and deployment strategies

### Data Science and Analytics
* Tableau or Data Visualization tools (e.g., D3.js, Matplotlib)
* Deep Learning frameworks (e.g., TensorFlow, PyTorch)
* Natural Language Processing (NLP) libraries (e.g., NLTK, spaCy)
* Data manipulation and analysis libraries (e.g., Pandas, NumPy)

### Programming Languages
* Java or JavaScript for web development
* Kotlin or other modern programming languages for Android or iOS development

### Database Management
* MongoDB or other NoSQL databases
* PostgreSQL or other relational databases

### DevOps and Testing
* Containerization (e.g., Docker)
* CI/CD pipelines (e.g., Jenkins, GitLab CI/CD)
* Testing frameworks (e.g., Pytest, JUnit)

### Note:
These are general suggestions based on common job requirements. It's essential to focus on the skills that align with your interests and the specific job postings you're targeting.

As a Computer Engineering graduate with a focus on Hardware and Systems, you may want to consider exploring more on Cloud Computing, DevOps, and Database Management. With your ongoing studies in Data Science, you're already on the right track. Adding skills related to Data Science and Analytics, such as Deep Learning and NLP, may be beneficial.

### Model stability and execution time


#### ChatGPT


In [23]:
for _ in range(3):
    start_time = time.time()

    job_market_assistant = JobMarketAssistant(
        model=chat_gpt,
        cover_letter_prompt="Write a cover letter for this job. Do not use too long and unnecessarily sophisticated words.\n\nMy CV:\n{cv}\n\nJob Posting:\n{job_desc}",
        job_title_prompt="Summarize the job title based on the following CV in maximum 10 words:\n\n{cv}\n\nProvide only the job title as plain text without any quotes and punctuation.",
        key_points_prompt="Job Posting:\n\n{job_desc}\n\nGenerate key points from these job posting (focus on skils and requirements) sorted from the most important to the least?",
        missing_skills_prompt="This is my CV:\n\n{cv}\n\nReturn me list of skills that I should learn and I don't have in my CV to become more employable for jobs with requirements listed below sorted from the most important to the least. The list has to be in Markdown format. Job postings:\n\n",
    )

    cover_letter = job_market_assistant.get_cover_letter(
        file_job_postings.iloc[0], cv_1
    )

    missing_skills = job_market_assistant.get_missing_skills(file_job_postings, cv_1, dream_job1)

    display(Markdown(f"##### Execution time: {time.time() - start_time} seconds"))

### Cover letter for Applied Data Scientist at dunnhumby:
[Your Name]
[Address]
[City, State ZIP Code]
[Email Address]
[Today’s Date]

[Employer’s Name]
[Company Name]
[Address]
[City, State ZIP Code]

Dear [Employer’s Name],

I am writing to express my interest in the Applied Data Scientist position at dunnhumby. With a degree in Artificial Intelligence and more than two years of experience in C++ development, I am confident in my abilities to excel in this role.

My education has equipped me with a strong foundation in Automatics and Robotics, as well as Finance and Accounting. However, my passion for AI led me to shift my focus and complete my degree in Artificial Intelligence, where I have gained a deep understanding and practical experience in courses such as Data Visualization, NLP, ML, DL, BigData, Information Retrieval, Computer Vision, and Data Mining.

In addition to my academic background, I have also been actively involved in the academic circle GHOST, where I currently hold a position in the section organizing the GHOST conference on ML. This experience has not only sharpened my skills in AI/ML but also taught me the value of teamwork, communication, and problem-solving.

I have also gained valuable experience through my employment history, where I have worked as a C++ Developer at CAD Projekt and as a cook and factory worker in the past. These roles have honed my programming skills in Python, C++, SQL, R, and Java, as well as my ability to work in a fast-paced environment and manage multiple tasks simultaneously.

What particularly interests me about dunnhumby is its mission to put customers first and its use of data science to drive growth and reimagine businesses. I am intrigued by the opportunity to apply my problem-solving skills and expertise in machine learning to deliver impactful solutions for clients.

Furthermore, I share dunnhumby's commitment to diversity and inclusion, which is evident in the company's thriving networks and flexible working options. I value a work-life balance and appreciate the opportunity to discuss agile working arrangements during the hiring process.

Thank you for considering my application. I am excited about the possibility of joining your dynamic team and contributing my skills to dunnhumby's mission. Please do not hesitate to contact me to discuss my qualifications further.

Sincerely,
[Your Name]

/home/krzychu/anaconda3/envs/nlp_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Postings best matching job title AI Engineer intern working on NLP problems without using some Internet Applications languages

,job_title,company,job_desc,location,score
0,Machine Learning Engineer,Connex One,Join us in pushing the boundaries in Conversat...,"Manchester Area, United Kingdom",0.745181
1,Machine Learning Engineer,Speechmatics,Speechmatics is a cutting-edge applied AI Rese...,"Cambridge, England, United Kingdom",0.742900
2,Machine Learning Engineer,Speechmatics,We aim to get you onboarded and started on som...,"Cambridge, England, United Kingdom",0.730420
3,Machine Learning Engineer,Stackfix,"About Us\n\nAt Stackfix, we’re transforming ho...","London, England, United Kingdom",0.720131
4,Artificial Intelligence Engineer,Bondex,About The Company\n\nWe are an early-stage (Se...,Greater Portsmouth Area,0.714325
5,Machine Learning Engineer - ML Ops,Builder.ai,Description\n\nAbout Builder.ai\n\nWe’re on a ...,"London, England, United Kingdom",0.707100
6,AI Integrations Engineer (Remote),Telnyx,About Telnyx\n\nTelnyx is an industry leader t...,"Dublin, County Dublin, Ireland",0.704437


### Missing skills:
1. Experience with cloud computing platforms (AWS, Azure, etc.)
2. Knowledge of natural language processing (NLP) and speech recognition technologies
3. Familiarity with data engineering and data architecture
4. Understanding of Agile methodologies and project management
5. Experience with deep learning frameworks (TensorFlow, PyTorch, etc.)
6. Knowledge of blockchain technology and its application to AI
7. Understanding of devOps and automation tools (Git, Jenkins, etc.)
8. Familiarity with chatbot development and conversational AI
9. Experience with data visualization tools (Tableau, PowerBI, etc.)
10. Understanding of ethical and responsible AI practices
11. Knowledge of reinforcement learning algorithms
12. Experience with biotech or healthcare industry
13. Familiarity with virtual and augmented reality technologies
14. Understanding of IoT and sensor data processing
15. Knowledge of financial analytics and risk management.

...

/home/krzychu/anaconda3/envs/nlp_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Postings best matching job title AI Engineer intern working on NLP problems without using some Internet Applications languages

,job_title,company,job_desc,location,score
0,Machine Learning Engineer,Connex One,Join us in pushing the boundaries in Conversat...,"Manchester Area, United Kingdom",0.745181
1,Machine Learning Engineer,Speechmatics,Speechmatics is a cutting-edge applied AI Rese...,"Cambridge, England, United Kingdom",0.742900
2,Machine Learning Engineer,Speechmatics,We aim to get you onboarded and started on som...,"Cambridge, England, United Kingdom",0.730420
3,Machine Learning Engineer,Stackfix,"About Us\n\nAt Stackfix, we’re transforming ho...","London, England, United Kingdom",0.720131
4,Artificial Intelligence Engineer,Bondex,About The Company\n\nWe are an early-stage (Se...,Greater Portsmouth Area,0.714325
5,Machine Learning Engineer - ML Ops,Builder.ai,Description\n\nAbout Builder.ai\n\nWe’re on a ...,"London, England, United Kingdom",0.707100
6,AI Integrations Engineer (Remote),Telnyx,About Telnyx\n\nTelnyx is an industry leader t...,"Dublin, County Dublin, Ireland",0.704437


### Missing skills:
1. Experience with Machine Learning (ML) and Deep Learning (DL) algorithms and techniques
2. Knowledge of data visualization tools such as Tableau or Power BI
3. Familiarity with Natural Language Processing (NLP) and its applications
4. Understanding of Big Data concepts and experience with handling large datasets
5. Proficiency in cloud computing platforms such as AWS or Azure
6. Ability to develop and deploy AI and ML models in production environments
7. Experience with computer vision and image processing techniques
8. Knowledge of data mining and its applications in business and industry
9. Familiarity with software development methodologies, such as Agile or DevOps
10. Understanding of statistical analysis and data modeling techniques
11. Experience with programming languages commonly used in AI such as TensorFlow or PyTorch
12. Knowledge of data warehousing and ETL (Extract, Transform, Load) processes
13. Familiarity with database management systems and SQL queries
14. Understanding of natural language understanding and generation techniques
15. Knowledge of data analytics and data engineering principles
16. Experience with data cleaning and preprocessing techniques
17. Understanding of data security and privacy in AI applications
18. Familiarity with blockchain technology and its potential in AI applications
19. Knowledge of reinforcement learning and its applications in AI
20. Experience with developing and deploying AI solutions on mobile platforms.

##### Execution time: 38.40593767166138 seconds

#### Gemini


In [30]:
start_time = time.time()

job_market_assistant = JobMarketAssistant(
    model=gemini,
    cover_letter_prompt="Write a cover letter for this job. Do not use too long and unnecessarily sophisticated words.\n\nMy CV:\n{cv}\n\nJob Posting:\n{job_desc}",
    job_title_prompt="Summarize the job title based on the following CV in maximum 10 words:\n\n{cv}\n\nProvide only the job title as plain text without any quotes and punctuation.",
    key_points_prompt="Job Posting:\n\n{job_desc}\n\nGenerate key points from these job posting (focus on skils and requirements) sorted from the most important to the least?",
    missing_skills_prompt="This is my CV:\n\n{cv}\n\nReturn me list of skills that I should learn and I don't have in my CV to become more employable for jobs with requirements listed below sorted from the most important to the least. The list has to be in Markdown format. Job postings:\n\n",
)

cover_letter = job_market_assistant.get_cover_letter(
    file_job_postings.iloc[0], cv_1
)

missing_skills = job_market_assistant.get_missing_skills(file_job_postings, cv_1, dream_job1)

display(Markdown(f"##### Execution time: {time.time() - start_time} seconds"))

### Cover letter for Applied Data Scientist at dunnhumby:
## Cover Letter for Applied Data Scientist at dunnhumby

**Dear Hiring Manager,**

I am writing to express my keen interest in the Applied Data Scientist position at dunnhumby. As a highly motivated and results-oriented individual with a strong background in Artificial Intelligence, I am confident that I possess the skills and experience necessary to excel in this role.

**My Qualifications:**

* **Education:**
    * Currently pursuing a Master's degree in Artificial Intelligence (expected graduation year: 2025).
    * Completed coursework in Data Visualization, NLP, ML, DL, Big Data, Information Retrieval, Computer Vision, and Data Mining.
    * Participated in an Erasmus exchange program during the summer semester of 2022.
    * Actively involved in the academic circle GHOST, focusing on AI/ML, and currently organizing the GHOST conference on ML.
* **Skills:**
    * Proficient in Python, C++,

/home/krzychu/anaconda3/envs/nlp_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Postings best matching job title AI Engineer intern working on NLP problems without using some Internet Applications languages

,job_title,company,job_desc,location,score
0,Machine Learning Engineer,Connex One,Join us in pushing the boundaries in Conversat...,"Manchester Area, United Kingdom",0.745181
1,Machine Learning Engineer,Speechmatics,Speechmatics is a cutting-edge applied AI Rese...,"Cambridge, England, United Kingdom",0.742900
2,Machine Learning Engineer,Speechmatics,We aim to get you onboarded and started on som...,"Cambridge, England, United Kingdom",0.730420
3,Machine Learning Engineer,Stackfix,"About Us\n\nAt Stackfix, we’re transforming ho...","London, England, United Kingdom",0.720131
4,Artificial Intelligence Engineer,Bondex,About The Company\n\nWe are an early-stage (Se...,Greater Portsmouth Area,0.714325
5,Machine Learning Engineer - ML Ops,Builder.ai,Description\n\nAbout Builder.ai\n\nWe’re on a ...,"London, England, United Kingdom",0.707100
6,AI Integrations Engineer (Remote),Telnyx,About Telnyx\n\nTelnyx is an industry leader t...,"Dublin, County Dublin, Ireland",0.704437


### Missing skills:
## Skills to Learn for Employability

Based on your CV and the provided job postings, here's a list of skills you could learn to improve your employability, sorted from most to least important:

**1. Deep Learning Frameworks:**

* **TensorFlow:** Open-source framework for numerical computation and large-scale machine learning.
* **PyTorch:** Python-based framework for building and deploying deep neural networks.
* **Keras:** High-level API for building and training deep learning models.

**2. Cloud Computing Platforms:**

* **AWS:** Amazon Web Services, offering a wide range of cloud computing services.
* **Azure:** Microsoft's cloud platform, providing various services for building and deploying AI applications.
* **GCP:** Google Cloud Platform, offering AI-specific services like Vertex AI and BigQuery ML.

**3. Advanced Data Analysis Techniques:**

* **Time Series Analysis:** Analyzing data with a time component, like stock

##### Execution time: 27.03601384162903 seconds

In [35]:
start_time = time.time()

job_market_assistant = JobMarketAssistant(
    model=gemini,
    cover_letter_prompt="Write a cover letter for this job. Do not use too long and unnecessarily sophisticated words.\n\nMy CV:\n{cv}\n\nJob Posting:\n{job_desc}",
    job_title_prompt="Summarize the job title based on the following CV in maximum 10 words:\n\n{cv}\n\nProvide only the job title as plain text without any quotes and punctuation.",
    key_points_prompt="Job Posting:\n\n{job_desc}\n\nGenerate key points from these job posting (focus on skils and requirements) sorted from the most important to the least?",
    missing_skills_prompt="This is my CV:\n\n{cv}\n\nReturn me list of skills that I should learn and I don't have in my CV to become more employable for jobs with requirements listed below sorted from the most important to the least. The list has to be in Markdown format. Job postings:\n\n",
)

cover_letter = job_market_assistant.get_cover_letter(
    file_job_postings.iloc[0], cv_1
)

missing_skills = job_market_assistant.get_missing_skills(file_job_postings, cv_1, dream_job1)

display(Markdown(f"##### Execution time: {time.time() - start_time} seconds"))

### Cover letter for Applied Data Scientist at dunnhumby:
## Dear [Hiring Manager name],

I am writing to express my keen interest in the Applied Data Scientist position at dunnhumby, as advertised on [website name]. With my strong academic background in Artificial Intelligence and practical experience in data analysis and programming, I am confident that I possess the skills and qualifications necessary to excel in this role.

My academic journey has equipped me with a solid foundation in data science principles and techniques. I am currently in my third year of a Bachelor's degree in Artificial Intelligence, where I have gained in-depth knowledge of Data Visualization, NLP, Machine Learning, Deep Learning, Big Data, Information Retrieval, Computer Vision, and Data Mining. Additionally, I have completed courses in Statistics, Mathematics, and Physics, providing me with a strong understanding of the theoretical underpinnings of data analysis.

Beyond academics, I have actively sought opportunities to apply my knowledge in real-world settings. During my internship as a C++ Developer at CAD Projekt, I gained valuable

/home/krzychu/anaconda3/envs/nlp_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Postings best matching job title AI Engineer intern working on NLP problems without using some Internet Applications languages

,job_title,company,job_desc,location,score
0,Machine Learning Engineer,Connex One,Join us in pushing the boundaries in Conversat...,"Manchester Area, United Kingdom",0.745181
1,Machine Learning Engineer,Speechmatics,Speechmatics is a cutting-edge applied AI Rese...,"Cambridge, England, United Kingdom",0.742900
2,Machine Learning Engineer,Speechmatics,We aim to get you onboarded and started on som...,"Cambridge, England, United Kingdom",0.730420
3,Machine Learning Engineer,Stackfix,"About Us\n\nAt Stackfix, we’re transforming ho...","London, England, United Kingdom",0.720131
4,Artificial Intelligence Engineer,Bondex,About The Company\n\nWe are an early-stage (Se...,Greater Portsmouth Area,0.714325
5,Machine Learning Engineer - ML Ops,Builder.ai,Description\n\nAbout Builder.ai\n\nWe’re on a ...,"London, England, United Kingdom",0.707100
6,AI Integrations Engineer (Remote),Telnyx,About Telnyx\n\nTelnyx is an industry leader t...,"Dublin, County Dublin, Ireland",0.704437


### Missing skills:
## Skills to Learn for Employability

Based on your CV and the provided job postings, here's a list of skills you could learn to improve your employability:

**Most Important:**

* **Deep Learning Frameworks:** TensorFlow, PyTorch, Keras (These are mentioned in several job postings and are becoming increasingly important in the field of AI.)
* **Cloud Computing Platforms:** AWS, Azure, GCP (These are mentioned in several job postings and are becoming increasingly important for deploying and scaling AI models.)
* **Advanced Statistics and Probability:** Bayesian statistics, Markov Chain Monte Carlo methods (These are mentioned in several job postings and are important for understanding and building complex AI models.)
* **Computer Vision:** Object detection, image segmentation, image classification (This is mentioned in several job postings and is a growing field within AI.)
* **Natural Language Processing:** Text summarization, machine translation, sentiment analysis (This is mentioned in several job postings and is another growing field within AI.)

##### Execution time: 37.81937026977539 seconds

In [34]:
start_time = time.time()

job_market_assistant = JobMarketAssistant(
    model=gemini,
    cover_letter_prompt="Write a cover letter for this job. Do not use too long and unnecessarily sophisticated words.\n\nMy CV:\n{cv}\n\nJob Posting:\n{job_desc}",
    job_title_prompt="Summarize the job title based on the following CV in maximum 10 words:\n\n{cv}\n\nProvide only the job title as plain text without any quotes and punctuation.",
    key_points_prompt="Job Posting:\n\n{job_desc}\n\nGenerate key points from these job posting (focus on skils and requirements) sorted from the most important to the least?",
    missing_skills_prompt="This is my CV:\n\n{cv}\n\nReturn me list of skills that I should learn and I don't have in my CV to become more employable for jobs with requirements listed below sorted from the most important to the least. The list has to be in Markdown format. Job postings:\n\n",
)

cover_letter = job_market_assistant.get_cover_letter(
    file_job_postings.iloc[0], cv_1
)

missing_skills = job_market_assistant.get_missing_skills(file_job_postings, cv_1, dream_job1)

display(Markdown(f"##### Execution time: {time.time() - start_time} seconds"))

### Cover letter for Applied Data Scientist at dunnhumby:
## Dear [Hiring Manager name],

I am writing to express my keen interest in the Applied Data Scientist position at dunnhumby, as advertised on [website name]. With my strong background in Artificial Intelligence, Data Science, and programming, I am confident that I possess the skills and experience necessary to excel in this role and contribute significantly to your team.

**My passion for data science stems from my desire to use technology to solve complex problems and make a tangible impact on the world.** During my studies in Artificial Intelligence, I have gained a deep understanding of machine learning, deep learning, data visualization, and natural language processing. I have also honed my programming skills in Python, C++, SQL, R, and Java, enabling me to effectively analyze and interpret large datasets.

**My professional experience as a C++ Developer at CAD Projekt has further strengthened my ability to apply data science principles in a real-world setting.** In this role, I have been responsible for developing and implementing software solutions

/home/krzychu/anaconda3/envs/nlp_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Postings best matching job title AI Engineer intern working on NLP problems without using some Internet Applications languages

,job_title,company,job_desc,location,score
0,Machine Learning Engineer,Connex One,Join us in pushing the boundaries in Conversat...,"Manchester Area, United Kingdom",0.745181
1,Machine Learning Engineer,Speechmatics,Speechmatics is a cutting-edge applied AI Rese...,"Cambridge, England, United Kingdom",0.742900
2,Machine Learning Engineer,Speechmatics,We aim to get you onboarded and started on som...,"Cambridge, England, United Kingdom",0.730420
3,Machine Learning Engineer,Stackfix,"About Us\n\nAt Stackfix, we’re transforming ho...","London, England, United Kingdom",0.720131
4,Artificial Intelligence Engineer,Bondex,About The Company\n\nWe are an early-stage (Se...,Greater Portsmouth Area,0.714325
5,Machine Learning Engineer - ML Ops,Builder.ai,Description\n\nAbout Builder.ai\n\nWe’re on a ...,"London, England, United Kingdom",0.707100
6,AI Integrations Engineer (Remote),Telnyx,About Telnyx\n\nTelnyx is an industry leader t...,"Dublin, County Dublin, Ireland",0.704437


### Missing skills:
## Skills to Learn for Employability in AI/ML Jobs

Based on your CV and the provided job postings, here's a list of skills you could learn to improve your employability for AI/ML jobs, sorted from most to least important:

**1. Deep Learning Frameworks:**

* **TensorFlow:** Open-source framework for numerical computation and large-scale machine learning.
* **PyTorch:** Python-based framework for building and training deep neural networks.
* **Keras:** High-level API for building and training deep learning models, running on top of TensorFlow or Theano.

**2. Advanced Machine Learning Techniques:**

* **Ensemble Methods:** Combining multiple models to improve predictive performance.
* **Bayesian Optimization:** Efficiently finding the best hyperparameters for your models.
* **Reinforcement Learning:** Training agents to make decisions in complex environments.

**3. Cloud Computing Platforms:**

* **AWS SageMaker:** Managed service for building,

##### Execution time: 27.84250545501709 seconds

#### Llama


In [26]:
for _ in range(3):
    start_time = time.time()

    job_market_assistant = JobMarketAssistant(
        model=llama,
        cover_letter_prompt="Write a cover letter for this job. Do not use too long and unnecessarily sophisticated words.\n\nMy CV:\n{cv}\n\nJob Posting:\n{job_desc}",
        job_title_prompt="Summarize the job title based on the following CV in maximum 10 words:\n\n{cv}\n\nProvide only the job title as plain text without any quotes and punctuation.",
        key_points_prompt="Job Posting:\n\n{job_desc}\n\nGenerate key points from these job posting (focus on skils and requirements) sorted from the most important to the least?",
        missing_skills_prompt="This is my CV:\n\n{cv}\n\nReturn me list of skills that I should learn and I don't have in my CV to become more employable for jobs with requirements listed below sorted from the most important to the least. The list has to be in Markdown format. Job postings:\n\n",
    )

    cover_letter = job_market_assistant.get_cover_letter(
        file_job_postings.iloc[0], cv_1
    )

    missing_skills = job_market_assistant.get_missing_skills(file_job_postings, cv_1, dream_job1)

    display(Markdown(f"##### Execution time: {time.time() - start_time} seconds"))

### Cover letter for Applied Data Scientist at dunnhumby:
Here is a cover letter tailored to the job posting:

Dear Hiring Manager,

I am excited to apply for the Applied Data Scientist position at dunnhumby. With a strong background in Artificial Intelligence and a passion for using data to drive business decisions, I am confident that my skills and experience make me an ideal candidate for this role.

As a current student of Artificial Intelligence, I have developed a solid foundation in programming languages such as Python, C++, SQL, and R. My experience as a C++ Developer at CAD Projekt has given me hands-on experience with software development and problem-solving. Additionally, my academic background in Automatics and Robotics, Finance and Accounting, and Artificial Intelligence has provided me with a strong understanding of data analysis and visualization.

I am particularly drawn to dunnhumby's mission to empower businesses to compete and thrive in the modern data-driven economy. As someone who is passionate about using data to drive business decisions, I am excited about the opportunity to join a team that is dedicated to putting the customer first.

In my current role as a C++ Developer, I have developed strong analytical and problem-solving skills, which I believe would be valuable in this role. I am also confident in my ability to communicate complex ideas effectively and work collaboratively as part of a team.

I am excited about the opportunity to join a company that values diversity and inclusion and offers a comprehensive rewards package and flexible working options. I am confident that my skills, experience, and passion for data science make me an ideal candidate for this role.

Thank you for considering my application. I would welcome the opportunity to discuss my application and how I can contribute to the success of dunnhumby.

Sincerely,
[Your Name]

/home/krzychu/anaconda3/envs/nlp_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Postings best matching job title AI Engineer intern working on NLP problems without using some Internet Applications languages

,job_title,company,job_desc,location,score
0,Machine Learning Engineer,Connex One,Join us in pushing the boundaries in Conversat...,"Manchester Area, United Kingdom",0.745181
1,Machine Learning Engineer,Speechmatics,Speechmatics is a cutting-edge applied AI Rese...,"Cambridge, England, United Kingdom",0.742900
2,Machine Learning Engineer,Speechmatics,We aim to get you onboarded and started on som...,"Cambridge, England, United Kingdom",0.730420
3,Machine Learning Engineer,Stackfix,"About Us\n\nAt Stackfix, we’re transforming ho...","London, England, United Kingdom",0.720131
4,Artificial Intelligence Engineer,Bondex,About The Company\n\nWe are an early-stage (Se...,Greater Portsmouth Area,0.714325
5,Machine Learning Engineer - ML Ops,Builder.ai,Description\n\nAbout Builder.ai\n\nWe’re on a ...,"London, England, United Kingdom",0.707100
6,AI Integrations Engineer (Remote),Telnyx,About Telnyx\n\nTelnyx is an industry leader t...,"Dublin, County Dublin, Ireland",0.704437


### Missing skills:
To help you become more employable for the jobs with specific requirements, I've analyzed the job postings and identified the skills you may want to consider learning. Here's the list, sorted from most important to least important:

**Most Important:**

* **Cloud Computing**: Many companies require experience with cloud platforms like AWS, Azure, or Google Cloud. Familiarity with cloud-based technologies can be a significant plus.
* **Data Analysis**: As you're already working as a C++ Developer, learning data analysis skills such as Pandas, NumPy, or Tableau can help you understand data-driven development better.

**Important:**

* **Web Development**: Knowing a front-end programming language like HTML5, CSS3, and JavaScript can be valuable in many industries. You can start with frameworks like React, Angular, or Vue.js.
* **Cybersecurity**: As AI is a prominent field, having basic knowledge of cybersecurity principles and how to apply them in AI/ML projects can be an asset.

**Useful:**

* **Databases**: Understanding database management systems like MongoDB, PostgreSQL, or MySQL is essential for storing and retrieving data in AI/ML projects.
* **Graphic Design**: Having basic graphic design skills using tools like Adobe Creative Cloud can be helpful in creating visualizations and presentations.

**Nice to Have:**

* **Reinforcement Learning**: As you're already studying Artificial Intelligence, learning Reinforcement Learning can be a valuable addition to your skillset.
* **Deep Learning**: Familiarizing yourself with popular deep learning frameworks like TensorFlow, PyTorch, or Keras can be beneficial.

Keep in mind that these are general suggestions, and the importance of each skill may vary depending on the specific job posting or company. Focus on the skills that align most closely with your current expertise and interests. Good luck with your learning endeavors!

...

/home/krzychu/anaconda3/envs/nlp_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Postings best matching job title AI Engineer intern working on NLP problems without using some Internet Applications languages

,job_title,company,job_desc,location,score
0,Machine Learning Engineer,Connex One,Join us in pushing the boundaries in Conversat...,"Manchester Area, United Kingdom",0.745181
1,Machine Learning Engineer,Speechmatics,Speechmatics is a cutting-edge applied AI Rese...,"Cambridge, England, United Kingdom",0.742900
2,Machine Learning Engineer,Speechmatics,We aim to get you onboarded and started on som...,"Cambridge, England, United Kingdom",0.730420
3,Machine Learning Engineer,Stackfix,"About Us\n\nAt Stackfix, we’re transforming ho...","London, England, United Kingdom",0.720131
4,Artificial Intelligence Engineer,Bondex,About The Company\n\nWe are an early-stage (Se...,Greater Portsmouth Area,0.714325
5,Machine Learning Engineer - ML Ops,Builder.ai,Description\n\nAbout Builder.ai\n\nWe’re on a ...,"London, England, United Kingdom",0.707100
6,AI Integrations Engineer (Remote),Telnyx,About Telnyx\n\nTelnyx is an industry leader t...,"Dublin, County Dublin, Ireland",0.704437


### Missing skills:
Based on the job postings, I've identified the skills you may want to consider learning to increase your employability. Here's the list in Markdown format, sorted from most important to least important:

### Top Priority

* **Cloud Computing**: With the increasing trend of cloud-based infrastructure, having skills in cloud computing platforms like AWS, Azure, or Google Cloud can be a significant advantage.
* **Containerization**: Knowledge of containerization tools like Docker can be beneficial for many job roles.

### High Priority

* **Cybersecurity**: As data security becomes a top concern, having skills in cybersecurity, such as penetration testing, vulnerability assessment, or security auditing, can be highly valuable.
* **Data Science**: With the increasing demand for data-driven decision-making, skills in data science, including data visualization, machine learning, and statistical modeling, can be highly desirable.
* **Web Development**: Familiarity with web development frameworks like React, Angular, or Vue.js can be beneficial for many job roles.

### Medium Priority

* **Big Data**: Knowledge of big data technologies like Hadoop, Spark, or NoSQL databases can be useful for many job roles.
* **Machine Learning**: While you already have some experience with ML, having a deeper understanding of machine learning algorithms and techniques can be beneficial.
* **DevOps**: Familiarity with DevOps tools like Jenkins, GitLab CI/CD, or Kubernetes can be useful for many job roles.

### Low Priority

* **Blockchain**: While blockchain technology is gaining popularity, it's not yet a requirement for most job postings.
* **Internet of Things (IoT)**: While IoT is an emerging field, it's not yet a priority for most job postings.

Remember that this is not an exhaustive list, and you should prioritize skills based on your interests, career goals, and job market demands.

##### Execution time: 107.84251976013184 seconds